# Classification

In this notebook, we will classify materials as metals or nonmetals. The dataset that we will use is built in the `dataset_preparation.ipynb` file. We will test many possible algorithms and to assess which one gives the better accuracy. The workflow is essentially the same for all algorithms: we perform a train test split; then perform a grid search evaluated against a 5-fold split of the training set as our validation set to find the best set of hyperparameters; finally, we evaluate the accuracy on the test data.

In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import multiprocessing
import xgboost as xgb #For parallel gradient boosting

In [5]:
#Dataset loading
df = pd.read_csv('gap_prediction.csv')

#Turning space group into a categorical variable
df["Space Group"] = df["Space Group"].astype('category')

#Building a dict that maps the space groups in unique integers
mapping_dict = dict(zip(df['Space Group'], df['Space Group'].cat.codes))

#Transforms the categorical space group to numbers
df['Space Group'] = df['Space Group'].map(mapping_dict)

#Target.; 1 if metal (gap==0); 0 otherwise
y = [1 if gap==0 else 0 for gap in df['gap']]
df.drop(['gap','Material','Unnamed: 0'], axis='columns', inplace=True)
X = df.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Models

## Logistic Regression

In [6]:
# Define the hyperparameters to tune and their possible values
param_grid = {
    'C': [0.001, 0.01,0.1,1,10],#, 0.1, 1, 10],  # Inverse of regularization strength
    'penalty': ['elasticnet', 'l1', 'l2']  # Regularization penalty (L1 or L2)
}

# Create a Logistc Regression classifier
lr_classifier = LogisticRegression(max_iter=50000,solver='saga')

# Use GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(lr_classifier, param_grid, cv=5, scoring='accuracy',n_jobs=-1)
scaler = StandardScaler().fit(X_train)
grid_search.fit(scaler.transform(X_train), y_train)
print(grid_search.best_params_)
best_params = grid_search.best_params_

/home/marcsgil/Desktop/MLPhysics/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
25 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/home/marcsgil/Desktop/MLPhysics/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/marcsgil/Desktop/MLPhysics/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/marcsgil/Desktop/MLPhysics/lib/python3.11/site-p

{'C': 0.1, 'penalty': 'l2'}


In [7]:
# Train the Logistc Regression classifier with the best hyperparameters
best_lr_classifier = LogisticRegression(**best_params, max_iter=10000)
best_lr_classifier.fit(scaler.transform(X_train), y_train)

# Evaluate the model on the test set
y_pred = best_lr_classifier.predict(scaler.transform(X_test))
accuracy_lr = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_lr)

# Perform Cross-Validation with the best hyperparameters
cv_scores_lr = cross_val_score(best_lr_classifier, X_train, y_train, cv=5, scoring='accuracy',n_jobs=-1)
print("Cross-Validation Scores:", cv_scores_lr)
print("Mean CV Accuracy:", np.mean(cv_scores_lr))

Test Accuracy: 0.7167182662538699
Cross-Validation Scores: [0.72340426 0.70736434 0.71317829 0.6996124  0.71899225]
Mean CV Accuracy: 0.7125103084281709


## Support Vector Machine

In [8]:
# Define the hyperparameters to tune and their possible values
param_grid = {
    'C': [0.1, 1, 10],              # Regularization parameter
    'kernel': ['linear', 'rbf'],    # Kernel type (linear or radial basis function)
    'gamma': ['scale', 'auto', 0.1]  # Kernel coefficient for 'rbf' kernel
}

# Create an SVM classifier
svm_classifier = SVC()

# Use GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='accuracy',n_jobs=-1)
scaler = StandardScaler().fit(X_train)
grid_search.fit(scaler.transform(X_train), y_train)
print(grid_search.best_params_)
best_params = grid_search.best_params_

{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [9]:
# Train the SVM classifier classifier with the best hyperparameters
best_svm_classifier = SVC(**best_params)
best_svm_classifier.fit(scaler.transform(X_train), y_train)

# Evaluate the model on the test set
y_pred = best_svm_classifier.predict(scaler.transform(X_test))
accuracy_svm = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_svm)

# Perform Cross-Validation with the best hyperparameters
cv_scores_svm = cross_val_score(best_svm_classifier, X_train, y_train, cv=5, scoring='accuracy',n_jobs=-1)
print("Cross-Validation Scores:", cv_scores_svm)
print("Mean CV Accuracy:", np.mean(cv_scores_svm))

Test Accuracy: 0.7724458204334366
Cross-Validation Scores: [0.60348162 0.60658915 0.6124031  0.62403101 0.60852713]
Mean CV Accuracy: 0.6110064024710239


## Decision Tree

In [10]:
# Define the hyperparameters to tune and their possible values
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum samples required to be at a leaf node
}

# Create a Random Forest classifier
dt_classifier = DecisionTreeClassifier()

# Use GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy',n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_params = grid_search.best_params_

{'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10}


In [11]:
#= Train the Decision Tree classifier with the best hyperparameters
best_dt_classifier = DecisionTreeClassifier(**best_params)
best_dt_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = best_dt_classifier.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_dt)

# Perform Cross-Validation with the best hyperparameters
cv_scores_dt = cross_val_score(best_dt_classifier, X_train, y_train, cv=5, scoring='accuracy',n_jobs=-1)
print("Cross-Validation Scores:", cv_scores_dt)
print("Mean CV Accuracy:", np.mean(cv_scores_dt))

Test Accuracy: 0.8436532507739938
Cross-Validation Scores: [0.82011605 0.79844961 0.80232558 0.80813953 0.8003876 ]
Mean CV Accuracy: 0.8058836759480006


## Random Forest

In [12]:
# Parameter Tuning with Cross-Validation
# Define the hyperparameters to tune and their possible values
param_grid = {
    'n_estimators': [100, 200, 300],      # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],     # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],    # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]       # Minimum samples required to be at a leaf node
}

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_jobs=-1)

# Use GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy',n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_params = grid_search.best_params_

{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}


In [13]:
# Train the Random Forest classifier with the best hyperparameters
best_rf_classifier = RandomForestClassifier(n_jobs=-1, **best_params)
best_rf_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = best_rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_rf)

# Perform Cross-Validation with the best hyperparameters
cv_scores_rf = cross_val_score(best_rf_classifier, X_train, y_train, cv=5, scoring='accuracy',n_jobs=-1)
print("Cross-Validation Scores:", cv_scores_rf)
print("Mean CV Accuracy:", np.mean(cv_scores_rf))

Test Accuracy: 0.8095975232198143
Cross-Validation Scores: [0.80851064 0.81976744 0.8120155  0.81007752 0.78682171]
Mean CV Accuracy: 0.8074385617681015


## Gradient Boosting

In [14]:
# Parameter Tuning with Cross-Validation
# Define the hyperparameters to tune and their possible values
param_grid = {
    'n_estimators': [50, 100, 200],      # Number of boosting stages to be used
    'learning_rate': [0.1, 0.2, 0.3, 0.4],  # Step size shrinks the contribution of each tree
    'max_depth': [5, 6, 7, 8]              # Maximum depth of each tree
}

# Create a Gradient Boosting classifier
xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist"
)

grid_search = GridSearchCV(xgb_model,param_grid,cv=5,scoring='accuracy',n_jobs=2)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_params = grid_search.best_params_

{'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 100}


In [15]:
# Train the Gradient Boosting classifier with the best hyperparameters
best_gb_classifier = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist", **best_params)
best_gb_classifier.fit(X_train, y_train,verbose=3)

# Evaluate the model on the test set
y_pred = best_gb_classifier.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_gb)

# Perform Cross-Validation with the best hyperparameters
cv_scores_gb = cross_val_score(best_gb_classifier, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", cv_scores_gb)
print("Mean CV Accuracy:", np.mean(cv_scores_gb))

Test Accuracy: 0.871517027863777
Cross-Validation Scores: [0.86653772 0.89341085 0.87596899 0.86821705 0.8624031 ]
Mean CV Accuracy: 0.8733075435203095


# Summary

In [16]:
df = pd.DataFrame(columns=['Algorithm', 'Test Accuracy', 'Mean CV Accuracy'])
df.loc[len(df)] = ['Logistic Regression', accuracy_lr, np.mean(cv_scores_lr)]
df.loc[len(df)] = ['Support Vector Machine', accuracy_svm, np.mean(cv_scores_svm)]
df.loc[len(df)] = ['Decision Tree', accuracy_dt, np.mean(cv_scores_dt)]
df.loc[len(df)] = ['Random Forrest', accuracy_rf, np.mean(cv_scores_rf)]
df.loc[len(df)] = ['Gradient Boosting', accuracy_gb, np.mean(cv_scores_gb)]
df.sort_values(by='Mean CV Accuracy', ascending=False)

,Algorithm,Test Accuracy,Mean CV Accuracy
4,Gradient Boosting,0.871517,0.873308
3,Random Forrest,0.809598,0.807439
2,Decision Tree,0.843653,0.805884
0,Logistic Regression,0.716718,0.712510
1,Support Vector Machine,0.772446,0.611006


We see that Gradient boosting was the best algorithm. We will use it to classify novel materials.

# Prediction of novel Materials

In [22]:
novel_df = pd.read_csv('gap_prediction_novel.csv')

novel_df["Space Group"] = novel_df["Space Group"].astype('category')
novel_df['Space Group'] = novel_df['Space Group'].map(mapping_dict)

novel_df.drop(['Unnamed: 0'], axis='columns', inplace=True)

X_novel = novel_df.drop(['Material'], axis='columns').to_numpy()

novel_df["Is Metal"] = best_gb_classifier.predict(X_novel)

#The next lines move "Is Metal" to the second column
columns = novel_df.columns.tolist()
columns.remove("Is Metal")
columns.insert(2, "Is Metal")
novel_df = novel_df[columns]

#We turn the Space Group back to letters
inverse_dict = {val:key for key,val in mapping_dict.items()}
novel_df["Space Group"] = novel_df["Space Group"].map(inverse_dict)

In [24]:
#10 predicted nonmetals
novel_df.sort_values(by='Is Metal',ascending=True).head(10)

,Material,Space Group,Is Metal,Z_mean,Electronegativity_mean,IonizationPotential_mean,ElectronAffinity_mean,HOMO_mean,LUMO_mean,r_s_orbital_mean,...,r_p_orbital_wstd,r_d_orbital_wstd,r_atomic_nonbonded_wstd,r_valence_lastorbital_wstd,r_covalent_wstd,Valence_wstd,PeriodicColumn_wstd,PeriodicColumn_upto18_wstd,NumberUnfilledOrbitals_wstd,Polarizability_wstd
2969,FeCl3,P3m1,0,21.5,2.495,-10.61635,-2.56435,-6.50895,2.19930,1.00770,...,0.282269,0.292034,0.021125,0.043828,0.028125,0.312500,0.3125,25.312500,2.812500,589.426531
3471,Ti2Cl3,P2/m,0,19.5,2.350,-10.58370,-1.69885,-6.42310,2.37915,1.11200,...,0.158671,0.178985,0.035594,0.014666,0.087464,2.340000,2.3400,43.940000,12.740000,1558.805274
3472,TiCl2,P2/m,0,19.5,2.350,-10.58370,-1.69885,-6.42310,2.37915,1.11200,...,0.169520,0.191223,0.038028,0.015668,0.093444,2.500000,2.5000,46.944444,13.611111,1665.390250
3473,TiCl3,P2/m,0,19.5,2.350,-10.58370,-1.69885,-6.42310,2.37915,1.11200,...,0.190710,0.215126,0.042781,0.017627,0.105125,2.812500,2.8125,52.812500,15.312500,1873.564031
3474,Ti2Br3,P2/m,0,28.5,2.250,-9.92245,-1.61315,-6.05500,2.62670,1.14845,...,0.111376,0.022032,0.012584,0.034487,0.041600,43.940000,2.3400,43.940000,12.740000,1281.290400
3475,TiBr2,P2/m,0,28.5,2.250,-9.92245,-1.61315,-6.05500,2.62670,1.14845,...,0.118992,0.023539,0.013444,0.036845,0.044444,46.944444,2.5000,46.944444,13.611111,1368.900000
3476,TiBr3,P2/m,0,28.5,2.250,-9.92245,-1.61315,-6.05500,2.62670,1.14845,...,0.133866,0.026481,0.015125,0.041451,0.050000,52.812500,2.8125,52.812500,15.312500,1540.012500
3477,Ti2I3,P2/m,0,37.5,2.100,-9.21605,-1.51440,-5.66770,2.03045,1.22395,...,0.055736,0.002993,0.000234,0.080289,0.011466,43.940000,2.3400,43.940000,12.740000,856.637600
3478,TiI2,P2/m,0,37.5,2.100,-9.21605,-1.51440,-5.66770,2.03045,1.22395,...,0.059547,0.003198,0.000250,0.085778,0.012250,46.944444,2.5000,46.944444,13.611111,915.211111
3479,TiI3,P2/m,0,37.5,2.100,-9.21605,-1.51440,-5.66770,2.03045,1.22395,...,0.066990,0.003598,0.000281,0.096501,0.013781,52.812500,2.8125,52.812500,15.312500,1029.612500


In [23]:
#10 predicted metals
novel_df.sort_values(by='Is Metal',ascending=False).head(10)

,Material,Space Group,Is Metal,Z_mean,Electronegativity_mean,IonizationPotential_mean,ElectronAffinity_mean,HOMO_mean,LUMO_mean,r_s_orbital_mean,...,r_p_orbital_wstd,r_d_orbital_wstd,r_atomic_nonbonded_wstd,r_valence_lastorbital_wstd,r_covalent_wstd,Valence_wstd,PeriodicColumn_wstd,PeriodicColumn_upto18_wstd,NumberUnfilledOrbitals_wstd,Polarizability_wstd
0,Sc2F3,Pmmn,1,15.0,2.670,-13.15365,-1.62310,-7.17485,0.76665,1.02750,...,0.405600,0.061058,0.138554,0.011130,0.331994,4.160000e+00,4.160000e+00,50.960000,16.640000,2774.431400
3169,MnF2,Pmmm,1,17.0,2.765,-13.33305,-2.55190,-7.75915,1.77415,0.88715,...,0.220325,0.120012,0.128444,0.000325,0.186778,7.888609e-31,7.888609e-31,27.777778,4.444444,1043.802778
3155,TiI3,Pmmm,1,37.5,2.100,-9.21605,-1.51440,-5.66770,2.03045,1.22395,...,0.066990,0.003598,0.000281,0.096501,0.013781,5.281250e+01,2.812500e+00,52.812500,15.312500,1029.612500
3156,Cr2F3,Pmmm,1,16.5,2.820,-13.29130,-2.42175,-7.78970,1.10385,0.91550,...,0.277067,0.101985,0.081536,0.001105,0.174824,2.600000e-01,2.600000e-01,31.460000,6.500000,1435.327400
3157,CrF2,Pmmm,1,16.5,2.820,-13.29130,-2.42175,-7.78970,1.10385,0.91550,...,0.296012,0.108959,0.087111,0.001181,0.186778,2.777778e-01,2.777778e-01,33.611111,6.944444,1533.469444
3158,CrF3,Pmmm,1,16.5,2.820,-13.29130,-2.42175,-7.78970,1.10385,0.91550,...,0.333014,0.122579,0.098000,0.001328,0.210125,3.125000e-01,3.125000e-01,37.812500,7.812500,1725.153125
3159,Cr2Cl3,Pmmm,1,20.5,2.410,-10.57820,-2.22255,-6.53005,2.70710,1.05255,...,0.109715,0.215164,0.020384,0.026210,0.035594,2.600000e-01,2.600000e-01,31.460000,6.500000,1046.074874
3160,CrCl2,Pmmm,1,20.5,2.410,-10.57820,-2.22255,-6.53005,2.70710,1.05255,...,0.117217,0.229876,0.021778,0.028002,0.038028,2.777778e-01,2.777778e-01,33.611111,6.944444,1117.601361
3161,CrCl3,Pmmm,1,20.5,2.410,-10.57820,-2.22255,-6.53005,2.70710,1.05255,...,0.131869,0.258611,0.024500,0.031502,0.042781,3.125000e-01,3.125000e-01,37.812500,7.812500,1257.301531
3162,Cr2Br3,Pmmm,1,29.5,2.310,-9.91695,-2.13685,-6.16195,2.95465,1.08900,...,0.071090,0.011586,0.004394,0.051302,0.009386,3.146000e+01,2.600000e-01,31.460000,6.500000,821.194400


In [34]:
print("Fraction of metals: %.2f" % (1 - novel_df["Is Metal"].sum()/novel_df["Is Metal"].shape[0]))

Fraction of metals: 0.38
